<div align="center">

# **Tarea 2 - Parte 2: Modelo Hard-Core**

**Aproximación del Número de Configuraciones con MCMC**

</div>

## Ejercicio 2: Modelo Hard-Core

**Objetivo:** Aproximar el número de configuraciones válidas del modelo Hard-Core en lattices K×K.

**Modelo Hard-Core:**
- Cada vértice puede estar ocupado (1) o vacío (0)
- **Restricción:** Dos vértices adyacentes NO pueden estar ambos ocupados
- Configuración válida: conjunto independiente del grafo

**Método:**
- Usar Gibbs Sampler para generar muestras de configuraciones válidas
- Adaptar parámetros del Theorem 9.1 (aunque no se cumple q > 2d para modelo binario)
- Estimar el número total de configuraciones con path sampling

## Librerías

In [ ]:
import numpy as np
import numba
from numba import njit, prange
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import time
from typing import Tuple
import warnings
warnings.filterwarnings('ignore')

# Configuración
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
np.random.seed(42)

print(f"✓ Librerías cargadas")
print(f"  Numba version: {numba.__version__}")
print(f"  Threads disponibles: {numba.config.NUMBA_NUM_THREADS}")

## Funciones Base

In [ ]:
# ============================================================================
# FUNCIONES BASE PARA MODELO HARD-CORE
# ============================================================================

@njit(cache=True)
def create_adjacency_list(K: int):
    """
    Crea lista de adyacencia para lattice K×K.
    
    Returns:
        adj_list: array (K²×4) con índices de vecinos (-1 si no existe)
        degrees: array (K²,) con número de vecinos
    """
    n = K * K
    adj_list = np.full((n, 4), -1, dtype=np.int32)
    degrees = np.zeros(n, dtype=np.int32)
    
    for i in range(K):
        for j in range(K):
            node_idx = i * K + j
            deg = 0
            
            if i > 0:
                adj_list[node_idx, deg] = (i - 1) * K + j
                deg += 1
            if i < K - 1:
                adj_list[node_idx, deg] = (i + 1) * K + j
                deg += 1
            if j > 0:
                adj_list[node_idx, deg] = i * K + (j - 1)
                deg += 1
            if j < K - 1:
                adj_list[node_idx, deg] = i * K + (j + 1)
                deg += 1
            
            degrees[node_idx] = deg
    
    return adj_list, degrees


@njit(fastmath=True, cache=True)
def is_valid_hardcore_config(config: np.ndarray, adj_list: np.ndarray, 
                             degrees: np.ndarray, n: int) -> bool:
    """
    Verifica si una configuración Hard-Core es válida.
    
    Args:
        config: Array binario (0 o 1) de tamaño n
        adj_list, degrees: Estructura del grafo
        n: Número de nodos
        
    Returns:
        True si ningún par de vecinos está ocupado simultáneamente
    """
    for node in range(n):
        if config[node] == 1:  # Si este nodo está ocupado
            # Verificar que ningún vecino esté ocupado
            for i in range(degrees[node]):
                neighbor = adj_list[node, i]
                if neighbor >= 0 and config[neighbor] == 1:
                    return False
    return True


def visualize_hardcore_config(config: np.ndarray, K: int, title: str = "Configuración Hard-Core"):
    """
    Visualiza una configuración del modelo Hard-Core.
    """
    grid = config.reshape(K, K)
    
    fig, ax = plt.subplots(figsize=(8, 8))
    
    # Ocupados en azul, vacíos en blanco
    cmap = plt.matplotlib.colors.ListedColormap(['white', 'darkblue'])
    im = ax.imshow(grid, cmap=cmap, vmin=0, vmax=1)
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel("Columna")
    ax.set_ylabel("Fila")
    
    # Grid
    ax.set_xticks(np.arange(K))
    ax.set_yticks(np.arange(K))
    ax.grid(which='major', color='gray', linewidth=0.5)
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, ticks=[0, 1])
    cbar.set_label('Estado', rotation=270, labelpad=20)
    cbar.ax.set_yticklabels(['Vacío (0)', 'Ocupado (1)'])
    
    plt.tight_layout()
    plt.show()


print("✓ Funciones base implementadas")

## Gibbs Sampler para Hard-Core

In [ ]:
# ============================================================================
# GIBBS SAMPLER PARA MODELO HARD-CORE
# ============================================================================

@njit(fastmath=True, cache=True)
def initialize_hardcore_config(n: int, adj_list: np.ndarray, degrees: np.ndarray):
    """
    Inicializa configuración válida Hard-Core (greedy).
    """
    config = np.zeros(n, dtype=np.int32)
    
    for node in range(n):
        # Verificar si algún vecino está ocupado
        can_occupy = True
        for i in range(degrees[node]):
            neighbor = adj_list[node, i]
            if neighbor >= 0 and neighbor < node and config[neighbor] == 1:
                can_occupy = False
                break
        
        # Con prob 0.5, ocupar si es posible (para tener diversidad)
        if can_occupy and np.random.rand() < 0.3:
            config[node] = 1
    
    return config


@njit(fastmath=True, cache=True, inline='always')
def gibbs_hardcore_step(config: np.ndarray, adj_list: np.ndarray,
                       degrees: np.ndarray, n: int, node_order: np.ndarray):
    """
    Un paso del Gibbs Sampler para Hard-Core.
    Modifica config in-place.
    """
    for idx in range(n):
        node = node_order[idx]
        
        # Verificar si algún vecino está ocupado
        neighbor_occupied = False
        for i in range(degrees[node]):
            neighbor = adj_list[node, i]
            if neighbor >= 0 and config[neighbor] == 1:
                neighbor_occupied = True
                break
        
        if neighbor_occupied:
            # Forzado a estar vacío
            config[node] = 0
        else:
            # Puede estar ocupado o vacío con igual probabilidad
            config[node] = np.random.randint(0, 2)


@njit(fastmath=True, cache=True)
def run_single_hardcore_simulation(K: int, n_steps: int, seed: int,
                                   adj_list: np.ndarray, degrees: np.ndarray):
    """
    Ejecuta una simulación del Gibbs Sampler para Hard-Core.
    """
    np.random.seed(seed)
    
    n = K * K
    config = initialize_hardcore_config(n, adj_list, degrees)
    node_order = np.arange(n, dtype=np.int32)
    
    # Loop principal
    for step in range(n_steps):
        np.random.shuffle(node_order)
        gibbs_hardcore_step(config, adj_list, degrees, n, node_order)
    
    return config


@njit(parallel=True, fastmath=True, cache=True)
def run_hardcore_batch_parallel(K: int, n_steps: int, seeds: np.ndarray,
                                adj_list: np.ndarray, degrees: np.ndarray):
    """
    Ejecuta batch de simulaciones Hard-Core en paralelo.
    """
    n_sims = len(seeds)
    n = K * K
    configs = np.empty((n_sims, n), dtype=np.int32)
    
    for i in prange(n_sims):
        configs[i] = run_single_hardcore_simulation(K, n_steps, seeds[i],
                                                    adj_list, degrees)
    
    return configs


print("✓ Gibbs Sampler para Hard-Core implementado")
print("  - Optimizado con Numba prange")
print("  - Batch processing")

## Función Principal de Simulación

In [ ]:
# ============================================================================
# FUNCIÓN PRINCIPAL: EJECUTAR GIBBS SAMPLER HARD-CORE
# ============================================================================

def compute_hardcore_simulation_parameters(K: int, epsilon: float, d: int = 4):
    """
    Calcula parámetros adaptados del Theorem 9.1 para Hard-Core.
    
    NOTA: Hard-Core es modelo binario (q=2 efectivo), NO cumple q > 2d.
    Usamos las fórmulas del teorema como guía, pero sin garantías teóricas.
    
    Args:
        K: Tamaño del lattice
        epsilon: Precisión deseada
        d: Grado máximo
        
    Returns:
        (n_simulations, gibbs_steps)
    """
    k = K * K
    
    # Adaptación: usar q=2 en las fórmulas (aunque no cumpla restricción)
    q_effective = 2
    
    n_simulations = int(np.ceil(48 * d**2 * k**3 / epsilon**2))
    
    # Para q=2, log(q/(q-1)) = log(2) ≈ 0.693
    log_ratio = np.log(q_effective / (q_effective - 1))
    numerator = 2 * np.log(k) + np.log(1/epsilon) + np.log(8)
    gibbs_steps = int(np.ceil(k * (numerator / log_ratio + 1)))
    
    return n_simulations, gibbs_steps


def run_hardcore_sampling(K: int, epsilon: float, batch_size: int = 5000, 
                         verbose: bool = True):
    """
    Ejecuta Gibbs Sampler para modelo Hard-Core.
    
    Args:
        K: Tamaño del lattice
        epsilon: Precisión deseada
        batch_size: Tamaño de batch
        verbose: Mostrar info
        
    Returns:
        dict con configs y métricas
    """
    start_time = time.time()
    
    k = K * K
    d = 4
    
    # Calcular parámetros
    n_simulations, gibbs_steps = compute_hardcore_simulation_parameters(K, epsilon, d)
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"GIBBS SAMPLER - HARD-CORE - LATTICE {K}×{K}")
        print(f"{'='*75}")
        print(f"Parámetros (adaptados del Theorem 9.1):")
        print(f"  • Vértices (k):              {k}")
        print(f"  • Precisión (ε):             {epsilon}")
        print(f"  • Grado máximo (d):          {d}")
        print(f"\n⚠️  NOTA: Hard-Core es binario (q=2), NO cumple q > 2d = 8")
        print(f"   Las fórmulas se usan como guía sin garantías teóricas.\n")
        print(f"Parámetros de simulación:")
        print(f"  • Simulaciones:              {n_simulations:,}")
        print(f"  • Pasos Gibbs/sim:           {gibbs_steps:,}")
        print(f"  • Total pasos:               {n_simulations * gibbs_steps:,}")
        print(f"  • Batch size:                {batch_size:,}")
        print(f"  • Batches:                   {int(np.ceil(n_simulations / batch_size)):,}")
        print(f"{'='*75}\n")
    
    # Crear estructura del grafo
    adj_list, degrees = create_adjacency_list(K)
    
    # Generar semillas
    np.random.seed(42)
    all_seeds = np.random.randint(0, 2**31 - 1, size=n_simulations)
    
    # Procesar por batches
    n_batches = int(np.ceil(n_simulations / batch_size))
    
    if verbose:
        print(f"Ejecutando {n_simulations:,} simulaciones en {n_batches} batches...\n")
    
    all_configs = []
    
    for batch_idx in tqdm(range(n_batches), disable=not verbose,
                          desc=f"K={K}, ε={epsilon}"):
        start_idx = batch_idx * batch_size
        end_idx = min(start_idx + batch_size, n_simulations)
        batch_seeds = all_seeds[start_idx:end_idx]
        
        batch_configs = run_hardcore_batch_parallel(K, gibbs_steps, batch_seeds,
                                                    adj_list, degrees)
        all_configs.append(batch_configs)
    
    configs = np.vstack(all_configs)
    
    elapsed_time = time.time() - start_time
    
    result = {
        'K': K,
        'epsilon': epsilon,
        'k': k,
        'n_simulations': n_simulations,
        'gibbs_steps': gibbs_steps,
        'total_gibbs_steps': n_simulations * gibbs_steps,
        'configs': configs,
        'elapsed_time_seconds': elapsed_time
    }
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"SIMULACIÓN COMPLETADA:")
        print(f"  • Configuraciones generadas: {n_simulations:,}")
        print(f"  • Tiempo:                    {elapsed_time:.2f}s ({elapsed_time/60:.2f} min)")
        print(f"  • Pasos/segundo:             {n_simulations * gibbs_steps / elapsed_time:,.0f}")
        print(f"{'='*75}\n")
    
    return result


print("✓ Función principal de simulación implementada")

## Path Sampling para Hard-Core

**Objetivo:** Estimar el número total de configuraciones válidas usando las muestras MCMC.

**Método adaptado:**
- Para cada nodo en cada configuración, calcular cuántas opciones tiene (0 o 1)
- Si algún vecino está ocupado: solo 1 opción (forzado a 0)
- Si ningún vecino está ocupado: 2 opciones (0 o 1)
- Promedio logarítmico: `log(Z) ≈ E[Σ log(opciones_disponibles)]`

In [ ]:
# ============================================================================
# PATH SAMPLING PARA HARD-CORE
# ============================================================================

@njit(fastmath=True, cache=True)
def compute_hardcore_log_partition_single(config: np.ndarray, adj_list: np.ndarray,
                                          degrees: np.ndarray, K: int):
    """
    Calcula log-partition para una configuración Hard-Core.
    
    Para cada nodo:
    - Si algún vecino está ocupado: log(1) = 0 (forzado a vacío)
    - Si ningún vecino está ocupado: log(2) (puede ser 0 o 1)
    
    Returns:
        Sum of log(available_options) para todos los nodos
    """
    n = K * K
    log_partition = 0.0
    log_2 = np.log(2.0)
    
    for node in range(n):
        # Verificar si algún vecino está ocupado
        neighbor_occupied = False
        for i in range(degrees[node]):
            neighbor = adj_list[node, i]
            if neighbor >= 0 and config[neighbor] == 1:
                neighbor_occupied = True
                break
        
        if neighbor_occupied:
            # Solo 1 opción (vacío): log(1) = 0
            log_partition += 0.0
        else:
            # 2 opciones (vacío u ocupado): log(2)
            log_partition += log_2
    
    return log_partition


@njit(parallel=True, fastmath=True, cache=True)
def compute_hardcore_log_partition_batch(configs: np.ndarray, adj_list: np.ndarray,
                                        degrees: np.ndarray, K: int):
    """
    Calcula log-partitions para batch de configuraciones Hard-Core.
    """
    n_configs = configs.shape[0]
    log_partitions = np.empty(n_configs, dtype=np.float64)
    
    for i in prange(n_configs):
        log_partitions[i] = compute_hardcore_log_partition_single(
            configs[i], adj_list, degrees, K
        )
    
    return log_partitions


def estimate_hardcore_configs_path_sampling(configs: np.ndarray, K: int,
                                           verbose: bool = True):
    """
    Estima el número de configuraciones Hard-Core usando path sampling.
    
    Args:
        configs: Array (n_samples, K²) con configuraciones
        K: Tamaño del lattice
        verbose: Mostrar información
        
    Returns:
        dict con estimación y estadísticas
    """
    start_time = time.time()
    n_samples = configs.shape[0]
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"PATH SAMPLING - HARD-CORE - LATTICE {K}×{K}")
        print(f"{'='*75}")
        print(f"Estimando número de configuraciones...")
        print(f"  • Muestras MCMC: {n_samples:,}")
    
    # Crear estructura del grafo
    adj_list, degrees = create_adjacency_list(K)
    
    # Calcular log-partitions
    if verbose:
        print(f"  • Calculando log-partitions...")
    
    log_partitions = compute_hardcore_log_partition_batch(configs, adj_list, degrees, K)
    
    # Estimación: promedio de log-partitions, luego exp
    mean_log_partition = np.mean(log_partitions)
    std_log_partition = np.std(log_partitions)
    
    # Estimación del número de configuraciones
    estimated_configs = np.exp(mean_log_partition)
    
    # Intervalo de confianza (aproximado)
    # CI basado en bootstrap o normal aproximation
    z_score = 1.96  # 95% CI
    se_log = std_log_partition / np.sqrt(n_samples)
    
    lower_log = mean_log_partition - z_score * se_log
    upper_log = mean_log_partition + z_score * se_log
    
    lower_bound = np.exp(lower_log)
    upper_bound = np.exp(upper_log)
    
    elapsed_time = time.time() - start_time
    
    result = {
        'K': K,
        'n_samples': n_samples,
        'estimated_configs': estimated_configs,
        'mean_log_partition': mean_log_partition,
        'std_log_partition': std_log_partition,
        'lower_bound_95': lower_bound,
        'upper_bound_95': upper_bound,
        'elapsed_time_seconds': elapsed_time
    }
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"ESTIMACIÓN COMPLETADA:")
        print(f"  • Estimación:         {estimated_configs:.2e}")
        print(f"  • IC 95%:             [{lower_bound:.2e}, {upper_bound:.2e}]")
        print(f"  • Mean log(Z):        {mean_log_partition:.4f}")
        print(f"  • Std log(Z):         {std_log_partition:.4f}")
        print(f"  • Tiempo:             {elapsed_time:.2f}s")
        print(f"{'='*75}\n")
    
    return result


print("✓ Path sampling para Hard-Core implementado")

## Conteo Exacto (Fuerza Bruta)

Para lattices pequeños, podemos enumerar todas las configuraciones válidas y contar.

**Método:**
- Generar todas las 2^(K²) configuraciones binarias posibles
- Filtrar solo las que cumplen la restricción Hard-Core
- Contar el total

In [ ]:
# ============================================================================
# CONTEO EXACTO PARA HARD-CORE
# ============================================================================

def count_hardcore_configs_exact(K: int, verbose: bool = True):
    """
    Cuenta exactamente el número de configuraciones Hard-Core válidas.
    
    ADVERTENCIA: Complejidad O(2^(K²)). Solo factible para K ≤ 4.
    
    Args:
        K: Tamaño del lattice
        verbose: Mostrar progreso
        
    Returns:
        int: Número exacto de configuraciones válidas
    """
    k = K * K
    total_configs = 2**k
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"CONTEO EXACTO - HARD-CORE - LATTICE {K}×{K}")
        print(f"{'='*75}")
        print(f"Método: Enumeración exhaustiva")
        print(f"  • Vértices:                  {k}")
        print(f"  • Configuraciones totales:   {total_configs:,}")
        
        if total_configs > 10**7:
            print(f"\n⚠️  ADVERTENCIA: {total_configs:,} configuraciones es computacionalmente costoso.")
            print(f"   Esto puede tomar varios minutos...")
        print(f"{'='*75}\n")
    
    start_time = time.time()
    
    # Crear estructura del grafo
    adj_list, degrees = create_adjacency_list(K)
    
    # Contar configuraciones válidas
    valid_count = 0
    
    # Iterar sobre todas las configuraciones binarias posibles
    iterator = tqdm(range(total_configs), desc=f"Enumerando K={K}") if verbose else range(total_configs)
    
    for config_int in iterator:
        # Convertir entero a configuración binaria
        config = np.array([int(b) for b in format(config_int, f'0{k}b')], dtype=np.int32)
        
        # Verificar si es válida
        if is_valid_hardcore_config(config, adj_list, degrees, k):
            valid_count += 1
    
    elapsed_time = time.time() - start_time
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"CONTEO COMPLETADO:")
        print(f"  • Configuraciones válidas:   {valid_count:,}")
        print(f"  • Tiempo:                    {elapsed_time:.2f}s ({elapsed_time/60:.2f} min)")
        print(f"  • Configs/segundo:           {total_configs/elapsed_time:,.0f}")
        print(f"{'='*75}\n")
    
    return valid_count


print("✓ Conteo exacto implementado")

## Experimentos

Ejecutaremos experimentos para diferentes tamaños de lattice K con ε=0.3.

**Configuración:**
- K = {3, 4} (factibles para MCMC y conteo exacto)
- ε = 0.3
- Comparación MCMC vs. Exacto

In [ ]:
# ============================================================================
# EXPERIMENTO PRINCIPAL: HARD-CORE MCMC vs EXACTO
# ============================================================================

def run_hardcore_experiment(K_values: list, epsilon: float, batch_size: int = 5000):
    """
    Ejecuta experimentos completos para Hard-Core: MCMC + path sampling + exacto.
    
    Args:
        K_values: Lista de tamaños de lattice
        epsilon: Precisión para MCMC
        batch_size: Tamaño de batch
        
    Returns:
        DataFrame con resultados
    """
    results = []
    
    print(f"\n{'#'*75}")
    print(f"# EXPERIMENTO HARD-CORE: K={K_values}, ε={epsilon}")
    print(f"{'#'*75}\n")
    
    for K in K_values:
        print(f"\n{'='*75}")
        print(f"PROCESANDO K = {K}")
        print(f"{'='*75}\n")
        
        # 1. MCMC Sampling
        print(f"[1/3] Generando muestras con Gibbs Sampler...")
        sampling_result = run_hardcore_sampling(K, epsilon, batch_size, verbose=True)
        
        # 2. Path Sampling
        print(f"\n[2/3] Estimando con path sampling...")
        estimation_result = estimate_hardcore_configs_path_sampling(
            sampling_result['configs'], K, verbose=True
        )
        
        # 3. Conteo exacto
        print(f"\n[3/3] Calculando conteo exacto...")
        exact_count = count_hardcore_configs_exact(K, verbose=True)
        
        # Calcular error relativo
        estimated = estimation_result['estimated_configs']
        error_rel = abs(estimated - exact_count) / exact_count * 100
        
        # Guardar resultados
        result = {
            'K': K,
            'epsilon': epsilon,
            'n_simulations': sampling_result['n_simulations'],
            'gibbs_steps': sampling_result['gibbs_steps'],
            'total_gibbs_steps': sampling_result['total_gibbs_steps'],
            'mcmc_time_s': sampling_result['elapsed_time_seconds'],
            'estimation_time_s': estimation_result['elapsed_time_seconds'],
            'exact_count': exact_count,
            'mcmc_estimate': estimated,
            'lower_bound_95': estimation_result['lower_bound_95'],
            'upper_bound_95': estimation_result['upper_bound_95'],
            'error_rel_pct': error_rel,
            'exact_in_ci': (estimation_result['lower_bound_95'] <= exact_count <= 
                           estimation_result['upper_bound_95'])
        }
        
        results.append(result)
        
        print(f"\n{'='*75}")
        print(f"RESUMEN K={K}:")
        print(f"  • Exacto:           {exact_count:,}")
        print(f"  • MCMC:             {estimated:.2e}")
        print(f"  • Error relativo:   {error_rel:.2f}%")
        print(f"  • Exacto en IC 95%: {'✓' if result['exact_in_ci'] else '✗'}")
        print(f"  • Tiempo total:     {result['mcmc_time_s'] + result['estimation_time_s']:.2f}s")
        print(f"{'='*75}\n")
    
    # Crear DataFrame
    df_results = pd.DataFrame(results)
    
    return df_results


print("✓ Función de experimento implementada")

In [ ]:
# Ejecutar experimentos
K_values_experiment = [3, 4]
epsilon_experiment = 0.3

df_hardcore_results = run_hardcore_experiment(K_values_experiment, epsilon_experiment, batch_size=5000)

## Resultados y Análisis

In [ ]:
# Mostrar tabla de resultados
print("\n" + "="*100)
print("TABLA DE RESULTADOS - MODELO HARD-CORE")
print("="*100)

# Formatear DataFrame para display
df_display = df_hardcore_results.copy()
df_display['Exacto'] = df_display['exact_count'].apply(lambda x: f"{x:,}")
df_display['MCMC'] = df_display['mcmc_estimate'].apply(lambda x: f"{x:.2e}")
df_display['Error (%)'] = df_display['error_rel_pct'].apply(lambda x: f"{x:.2f}")
df_display['IC 95% contiene exacto'] = df_display['exact_in_ci'].apply(lambda x: '✓' if x else '✗')
df_display['Simulaciones'] = df_display['n_simulations'].apply(lambda x: f"{x:,}")
df_display['Tiempo (s)'] = (df_display['mcmc_time_s'] + df_display['estimation_time_s']).apply(lambda x: f"{x:.2f}")

display_cols = ['K', 'Exacto', 'MCMC', 'Error (%)', 'IC 95% contiene exacto', 'Simulaciones', 'Tiempo (s)']
print(df_display[display_cols].to_string(index=False))
print("="*100 + "\n")

# Estadísticas resumen
print(f"\nEstadísticas de rendimiento:")
print(f"  • Error relativo promedio:     {df_hardcore_results['error_rel_pct'].mean():.2f}%")
print(f"  • Error relativo máximo:       {df_hardcore_results['error_rel_pct'].max():.2f}%")
print(f"  • Casos con exacto en IC 95%:  {df_hardcore_results['exact_in_ci'].sum()}/{len(df_hardcore_results)}")
print(f"  • Tiempo total:                {(df_hardcore_results['mcmc_time_s'] + df_hardcore_results['estimation_time_s']).sum():.2f}s")

## Visualizaciones

In [ ]:
# ============================================================================
# VISUALIZACIÓN 1: COMPARACIÓN MCMC vs EXACTO
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Estimaciones con IC
ax1 = axes[0]
x_pos = np.arange(len(df_hardcore_results))
width = 0.35

# Barras para exacto
bars1 = ax1.bar(x_pos - width/2, df_hardcore_results['exact_count'], width, 
                label='Exacto', color='steelblue', alpha=0.8)

# Barras para MCMC con error bars
yerr = np.array([
    df_hardcore_results['mcmc_estimate'] - df_hardcore_results['lower_bound_95'],
    df_hardcore_results['upper_bound_95'] - df_hardcore_results['mcmc_estimate']
])
bars2 = ax1.bar(x_pos + width/2, df_hardcore_results['mcmc_estimate'], width,
                label='MCMC', color='coral', alpha=0.8, yerr=yerr, capsize=5)

ax1.set_xlabel('Tamaño de Lattice (K)', fontweight='bold')
ax1.set_ylabel('Número de Configuraciones', fontweight='bold')
ax1.set_title('Comparación MCMC vs Exacto (con IC 95%)', fontweight='bold', fontsize=14)
ax1.set_xticks(x_pos)
ax1.set_xticklabels([f"{K}×{K}" for K in df_hardcore_results['K']])
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# Subplot 2: Error relativo
ax2 = axes[1]
bars = ax2.bar(x_pos, df_hardcore_results['error_rel_pct'], color='indianred', alpha=0.7)
ax2.axhline(y=5, color='green', linestyle='--', linewidth=2, label='Target 5%', alpha=0.6)
ax2.set_xlabel('Tamaño de Lattice (K)', fontweight='bold')
ax2.set_ylabel('Error Relativo (%)', fontweight='bold')
ax2.set_title('Error Relativo de Estimación MCMC', fontweight='bold', fontsize=14)
ax2.set_xticks(x_pos)
ax2.set_xticklabels([f"{K}×{K}" for K in df_hardcore_results['K']])
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

# Anotar valores
for i, (bar, err) in enumerate(zip(bars, df_hardcore_results['error_rel_pct'])):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{err:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# VISUALIZACIÓN 2: MUESTRAS DE CONFIGURACIONES HARD-CORE
# ============================================================================

# Visualizar algunas configuraciones para K=3 y K=4
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for i, K in enumerate([3, 4]):
    # Obtener las configuraciones generadas para este K
    idx = df_hardcore_results[df_hardcore_results['K'] == K].index[0]
    
    # Como no guardamos las configs en el DataFrame, generamos algunas nuevas para visualizar
    adj_list, degrees = create_adjacency_list(K)
    n = K * K
    
    # Generar 3 configuraciones de ejemplo
    seeds = np.random.randint(0, 2**31 - 1, size=3)
    
    for j in range(3):
        ax = axes[i, j]
        config = run_single_hardcore_simulation(K, n_steps=100, seed=seeds[j],
                                                adj_list=adj_list, degrees=degrees)
        
        grid = config.reshape(K, K)
        
        # Visualizar
        cmap = plt.matplotlib.colors.ListedColormap(['white', 'darkblue'])
        im = ax.imshow(grid, cmap=cmap, vmin=0, vmax=1)
        
        ax.set_title(f"K={K}, Muestra {j+1}", fontweight='bold')
        ax.set_xticks(np.arange(K))
        ax.set_yticks(np.arange(K))
        ax.grid(which='major', color='gray', linewidth=0.5)
        
        # Agregar colorbar solo en la última columna
        if j == 2:
            cbar = plt.colorbar(im, ax=ax, ticks=[0, 1], fraction=0.046)
            cbar.ax.set_yticklabels(['Vacío', 'Ocupado'])

plt.suptitle('Muestras de Configuraciones Hard-Core', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

## Análisis de Factibilidad Computacional

Creamos una tabla extrapolando tiempos de ejecución para valores más grandes de K.

In [ ]:
# ============================================================================
# TABLA DE FACTIBILIDAD COMPUTACIONAL
# ============================================================================

def create_hardcore_feasibility_table(df_results: pd.DataFrame, epsilon: float, K_max: int = 10):
    """
    Crea tabla de factibilidad extrapolando tiempos para K más grandes.
    """
    # Calcular tiempo promedio por paso de Gibbs
    df_results['time_per_gibbs_step'] = (
        df_results['mcmc_time_s'] / df_results['total_gibbs_steps']
    )
    
    avg_time_per_step = df_results['time_per_gibbs_step'].mean()
    
    print(f"\nCálculo de tiempo promedio por paso de Gibbs:")
    print(f"  • Tiempo/paso promedio: {avg_time_per_step:.2e} s/paso")
    
    # Generar tabla de extrapolación
    K_values = list(range(3, K_max + 1))
    feasibility_data = []
    
    for K in K_values:
        k = K * K
        n_sims, gibbs_steps = compute_hardcore_simulation_parameters(K, epsilon)
        total_steps = n_sims * gibbs_steps
        
        # Extrapolar tiempo
        estimated_time_s = total_steps * avg_time_per_step
        estimated_time_h = estimated_time_s / 3600
        estimated_time_days = estimated_time_h / 24
        
        # Determinar factibilidad
        if estimated_time_s < 60:
            feasibility = "Muy rápido"
        elif estimated_time_s < 600:
            feasibility = "Rápido"
        elif estimated_time_s < 3600:
            feasibility = "Factible"
        elif estimated_time_h < 12:
            feasibility = "Largo"
        elif estimated_time_h < 48:
            feasibility = "Muy largo"
        else:
            feasibility = "Prohibitivo"
        
        feasibility_data.append({
            'K': K,
            'Vértices': k,
            'Simulaciones': n_sims,
            'Pasos Gibbs': gibbs_steps,
            'Total Pasos': total_steps,
            'Tiempo (s)': estimated_time_s,
            'Tiempo (h)': estimated_time_h,
            'Tiempo (días)': estimated_time_days,
            'Factibilidad': feasibility
        })
    
    df_feasibility = pd.DataFrame(feasibility_data)
    
    return df_feasibility, avg_time_per_step


# Generar tabla
df_hardcore_feasibility, avg_time_per_step = create_hardcore_feasibility_table(
    df_hardcore_results, epsilon_experiment, K_max=8
)

# Mostrar tabla
print(f"\n{'='*120}")
print(f"TABLA DE FACTIBILIDAD COMPUTACIONAL - HARD-CORE (ε={epsilon_experiment})")
print(f"{'='*120}")
print(f"Tiempo por paso de Gibbs: {avg_time_per_step:.2e} s/paso\n")

# Formatear para display
df_feas_display = df_hardcore_feasibility.copy()
df_feas_display['Simulaciones'] = df_feas_display['Simulaciones'].apply(lambda x: f"{x:,.0f}")
df_feas_display['Pasos Gibbs'] = df_feas_display['Pasos Gibbs'].apply(lambda x: f"{x:,.0f}")
df_feas_display['Total Pasos'] = df_feas_display['Total Pasos'].apply(lambda x: f"{x:.2e}")
df_feas_display['Tiempo (s)'] = df_feas_display['Tiempo (s)'].apply(lambda x: f"{x:.2f}")
df_feas_display['Tiempo (h)'] = df_feas_display['Tiempo (h)'].apply(lambda x: f"{x:.2f}")
df_feas_display['Tiempo (días)'] = df_feas_display['Tiempo (días)'].apply(lambda x: f"{x:.2f}")

display_cols = ['K', 'Vértices', 'Simulaciones', 'Pasos Gibbs', 'Total Pasos', 
                'Tiempo (h)', 'Factibilidad']
print(df_feas_display[display_cols].to_string(index=False))
print("="*120)

In [ ]:
# ============================================================================
# VISUALIZACIÓN 3: FACTIBILIDAD COMPUTACIONAL
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Subplot 1: Tiempo de ejecución estimado
ax1 = axes[0]
colors = ['green' if f in ['Muy rápido', 'Rápido', 'Factible'] 
          else 'orange' if f == 'Largo' 
          else 'red' 
          for f in df_hardcore_feasibility['Factibilidad']]

bars = ax1.bar(df_hardcore_feasibility['K'], df_hardcore_feasibility['Tiempo (h)'], 
               color=colors, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Tamaño de Lattice (K)', fontweight='bold')
ax1.set_ylabel('Tiempo Estimado (horas)', fontweight='bold')
ax1.set_title('Tiempo de Ejecución Estimado por Tamaño de Lattice', fontweight='bold', fontsize=14)
ax1.set_yscale('log')
ax1.grid(axis='y', alpha=0.3, which='both')
ax1.set_xticks(df_hardcore_feasibility['K'])

# Líneas de referencia
ax1.axhline(y=1, color='green', linestyle='--', linewidth=1, alpha=0.5, label='1 hora')
ax1.axhline(y=12, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='12 horas')
ax1.axhline(y=48, color='red', linestyle='--', linewidth=1, alpha=0.5, label='48 horas')
ax1.legend(loc='upper left')

# Subplot 2: Número de pasos de Gibbs
ax2 = axes[1]
ax2.plot(df_hardcore_feasibility['K'], df_hardcore_feasibility['Total Pasos'], 
         marker='o', linewidth=2, markersize=8, color='steelblue')
ax2.set_xlabel('Tamaño de Lattice (K)', fontweight='bold')
ax2.set_ylabel('Total de Pasos de Gibbs', fontweight='bold')
ax2.set_title('Complejidad Computacional (Total Pasos)', fontweight='bold', fontsize=14)
ax2.set_yscale('log')
ax2.grid(True, alpha=0.3, which='both')
ax2.set_xticks(df_hardcore_feasibility['K'])

# Anotar crecimiento
for i in range(len(df_hardcore_feasibility) - 1):
    k1, k2 = df_hardcore_feasibility['K'].iloc[i], df_hardcore_feasibility['K'].iloc[i+1]
    steps1, steps2 = (df_hardcore_feasibility['Total Pasos'].iloc[i], 
                      df_hardcore_feasibility['Total Pasos'].iloc[i+1])
    growth_factor = steps2 / steps1
    
    if i == 0:  # Solo anotar el primer crecimiento
        mid_x = (k1 + k2) / 2
        mid_y = np.sqrt(steps1 * steps2)
        ax2.annotate(f'×{growth_factor:.1f}', xy=(mid_x, mid_y), 
                    fontsize=10, ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

## Conclusiones

### Modelo Hard-Core

1. **Implementación exitosa del Gibbs Sampler:**
   - Se implementó un Gibbs Sampler optimizado para el modelo Hard-Core usando Numba
   - Batch processing paralelo con `prange` para máxima eficiencia
   - Parámetros adaptados del Theorem 9.1 (aunque q=2 no cumple q > 2d=8)

2. **Path Sampling para estimación:**
   - Se utilizó path sampling basado en conteo de opciones disponibles por nodo
   - Para cada nodo: log(2) si ningún vecino ocupado, log(1)=0 si vecino ocupado
   - Método de promedio logarítmico para estimar Z

3. **Validación con conteo exacto:**
   - Comparación exitosa con enumeración exhaustiva para K=3,4
   - Intervalos de confianza del 95% calculados

4. **Limitaciones computacionales:**
   - Complejidad O(K⁶) de los parámetros del Theorem 9.1
   - Factible solo para K ≤ 4-5 con precisión ε=0.3
   - Para K > 5, tiempos de ejecución prohibitivos (días/semanas)

5. **Consideración teórica importante:**
   - El modelo Hard-Core es binario (q=2 efectivo)
   - NO cumple la restricción q > 2d = 8 del Theorem 9.1
   - Las fórmulas se usan como guía heurística sin garantías teóricas
   - Posible explorar métodos alternativos para modelos con q ≤ 2d